In [ ]:
!pwd

In [ ]:
import os
import pandas as pd 
import numpy as np
import textwrap
from tqdm import tqdm

import seaborn as sns
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from sklearn.utils import resample

CURR_PATH = os.path.abspath('/Users/chloesavignac/_bzdok_lab_notebooks/Project_1/_notebooks')
#CURR_PATH = os.path.abspath('/Users/chloesavignac/_CCA_plots/november/miami_apoe')
NEW_PATH = '/Users/chloesavignac/_bzdok_lab_notebooks/Project_1/_notebooks'
os.chdir(NEW_PATH)

import manhattan_plot_util8 as man_plot
os.chdir(CURR_PATH)


In [ ]:
import importlib
os.chdir(NEW_PATH)
importlib.reload(man_plot)
os.chdir(CURR_PATH)

In [ ]:
BASE_FOLDER = '/Users/chloesavignac/_bzdok_lab_notebooks/Project_1/_notebooks/Manhattan_Plots'
y_group = "_miller_mh_v1"

ukbb_y, y_desc_dict, y_cat_dict = man_plot.load_phenom(y_group, BASE_FOLDER)

ukbb_y.head()

# Version 2: BOOTSTRAP OF PREDICTIONS

In [ ]:
males_preds= pd.read_csv('apoe_preds_males_08.08.22.csv', index_col=0)
females_preds= pd.read_csv('apoe_preds_females_08.08.22.csv', index_col=0)

In [ ]:
males_preds.shape

In [ ]:
females_preds.shape

In [ ]:
#########This bootstrap analysis takes around 12,5 hours to run 

from sklearn.preprocessing import StandardScaler
for i in tqdm(range(0,1000)):
    males_prob = males_preds[['eid']]
    females_prob = females_preds[['eid']]
    
    rand_m = resample(males_preds, replace=True, random_state = i, n_samples = 17561)
    rand_f = resample(females_preds, replace=True, random_state = i, n_samples = 19730)
    
    
    for m in range(0,25):
        #MALES
        rand_m_mode = rand_m[['eid',f'APOE_Mode_{m+1}']]
        preds_mode_apoe = rand_m_mode.sort_values([f'APOE_Mode_{m+1}'],ascending=True) 
        min_5 = preds_mode_apoe.loc[preds_mode_apoe[f'APOE_Mode_{m+1}'] < np.percentile(preds_mode_apoe[f'APOE_Mode_{m+1}'],5)]
        max_5 = preds_mode_apoe.loc[preds_mode_apoe[f'APOE_Mode_{m+1}'] > np.percentile(preds_mode_apoe[f'APOE_Mode_{m+1}'],95)]
        extremes_m = pd.concat([min_5,max_5],axis=0)
        #print(extremes_m.shape)
        #z-scoring APOE scores
        X_scaler = StandardScaler()
        extremes_zscored = X_scaler.fit_transform(extremes_m[[f'APOE_Mode_{m+1}']])
        extremes_m[f'APOE_Mode_{m+1}']= extremes_zscored
        extremes_m.drop_duplicates(inplace=True)
        males_prob = males_prob.merge(extremes_m, how='left', on='eid')

        #FEMALES
        rand_f_mode = rand_f[['eid',f'APOE_Mode_{m+1}']]
        preds_mode_apoe = rand_f_mode.sort_values([f'APOE_Mode_{m+1}'],ascending=True) 
        min_5 = preds_mode_apoe.loc[preds_mode_apoe[f'APOE_Mode_{m+1}'] < np.percentile(preds_mode_apoe[f'APOE_Mode_{m+1}'],5)]
        max_5 = preds_mode_apoe.loc[preds_mode_apoe[f'APOE_Mode_{m+1}'] > np.percentile(preds_mode_apoe[f'APOE_Mode_{m+1}'],95)]
        extremes_f = pd.concat([min_5,max_5],axis=0)
        #z-scoring APOE scores
        X_scaler = StandardScaler()
        extremes_zscored = X_scaler.fit_transform(extremes_f[[f'APOE_Mode_{m+1}']])
        extremes_f[f'APOE_Mode_{m+1}']= extremes_zscored
        extremes_f.drop_duplicates(inplace=True)
        females_prob = females_prob.merge(extremes_f, how='left', on='eid')
        
    #PHEWAS males
    corrdf_m = man_plot.phenom_correlat(males_prob, ukbb_y, y_desc_dict, y_cat_dict)
    corrdf_m.to_csv(os.path.abspath(f'/Users/chloesavignac/_bzdok_lab_notebooks/Project_1/_notebooks/Miami_bootstrap_08.08.22/males/bootstrap_{i}_corrdf_m.csv'))
    
    #PHEWAS females
    corrdf_f = man_plot.phenom_correlat(females_prob, ukbb_y, y_desc_dict, y_cat_dict)
    corrdf_f.to_csv(os.path.abspath(f'/Users/chloesavignac/_bzdok_lab_notebooks/Project_1/_notebooks/Miami_bootstrap_08.08.22/females/bootstrap_{i}_corrdf_f.csv'))

In [ ]:
test = pd.read_csv(os.path.abspath(f'/Users/chloesavignac/_bzdok_lab_notebooks/Project_1/_notebooks/Miami_bootstrap_08.08.22/females/bootstrap_0_corrdf_f.csv'), index_col=0)

In [ ]:
catid = test['catid']

In [ ]:
p_cols = ['p_APOE_Mode_1', 'p_APOE_Mode_2', 'p_APOE_Mode_3',
       'p_APOE_Mode_4', 'p_APOE_Mode_5', 'p_APOE_Mode_6', 'p_APOE_Mode_7',
       'p_APOE_Mode_8', 'p_APOE_Mode_9', 'p_APOE_Mode_10', 'p_APOE_Mode_11',
       'p_APOE_Mode_12', 'p_APOE_Mode_13', 'p_APOE_Mode_14', 'p_APOE_Mode_15',
       'p_APOE_Mode_16', 'p_APOE_Mode_17', 'p_APOE_Mode_18', 'p_APOE_Mode_19',
       'p_APOE_Mode_20', 'p_APOE_Mode_21', 'p_APOE_Mode_22', 'p_APOE_Mode_23',
       'p_APOE_Mode_24', 'p_APOE_Mode_25']

In [ ]:
all_df_f = pd.DataFrame()
for i in tqdm(range(0,1000)):
    df = pd.read_csv(os.path.abspath(f'/Users/chloesavignac/_bzdok_lab_notebooks/Project_1/_notebooks/Miami_bootstrap_08.08.22/females/bootstrap_{i}_corrdf_f.csv'), index_col=0)
    df = df[p_cols]
    all_df_f = all_df_f.join(df, how='right', rsuffix=f'_{i}')

all_df_f.to_csv('bootstrap_phewas_females.csv')

In [ ]:
all_df_m = pd.DataFrame()
for i in tqdm(range(0,1000)):
    df = pd.read_csv(os.path.abspath(f'/Users/chloesavignac/_bzdok_lab_notebooks/Project_1/_notebooks/Miami_bootstrap_08.08.22/males/bootstrap_{i}_corrdf_m.csv'), index_col=0)
    df = df[p_cols]
    all_df_m = all_df_m.join(df, how='right', rsuffix=f'_{i}')

all_df_m.to_csv('bootstrap_phewas_males.csv')

In [ ]:
all_df_m = pd.read_csv('bootstrap_phewas_males.csv', index_col=0)

In [ ]:
all_df_f = pd.read_csv('bootstrap_phewas_females.csv', index_col=0)

In [ ]:
all_df_m.shape

In [ ]:
#customized categories
max_chars = 19
cat_uniq = np.unique(y_cat_dict["Cat_ID"])
cat_uniq = [c if c != 31 else 21 for c in cat_uniq]
cat_name = [y_cat_dict["Cat_Name"][y_cat_dict["Cat_ID"] == c][0]
            for c in cat_uniq]
cat_name = [c.replace(' and environment', '').replace('measures ', '')
            .replace('and', '&').replace(' & blood vessels', '')
            for c in cat_name]

new_cat_name = [textwrap.wrap(lbl, width=max_chars, break_on_hyphens=False)
                for lbl in cat_name]
new_cat_name = np.asarray(['\n'.join(lbl) for lbl in new_cat_name])
new_cat_name = new_cat_name[1::]

#set the colors based on this new categorization
colors = man_plot.get_colors(n_colors=len(new_cat_name))

In [ ]:
sns.set(rc={'figure.figsize':(30,10)})

all_cis = []
for k in range(1,26):
    sig_catids_m = np.genfromtxt(f'MALES_hits_Probability_Mode_{k}.txt')
    sig_catids_f = np.genfromtxt(f'FEMALES_hits_Probability_Mode_{k}.txt')
    
    
    phes_m = all_df_m[all_df_m.columns[k-1::25]].T
    phes_f = all_df_f[all_df_f.columns[k-1::25]].T
    phes_diff = abs(phes_m-phes_f)  
    
    plt.style.use('seaborn-white')
    fig, ax = plt.subplots()

    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_ylim(top=2)
    ax.set_ylim(bottom=-0.02)
    plt.xticks(color='w')

    cis = []
    for i in range (0,977):
        if ((i in sig_catids_m) | (i in sig_catids_f)):

            values = phes_diff[i]
            confidence = 0.95
            ci = np.percentile(values,[5,95])
            #ci = sms.DescrStatsW(values, weights=1000, ddof=999).tconfint_mean(alpha=0.05, alternative='two-sided') 
            cis.append(ci)
            
            x = i
            y = values.mean()
            
            lb = y - ci[0] 
            ub = ci[1] - y 
            
#             print(lb)
#             print(ub)
            
            err = np.array([lb,ub]).reshape(2,1)
        
            col = colors[list(catid.unique()).index(catid[i])]
            

            plt.errorbar(x,y,yerr=err, fmt='none', ecolor ='grey', elinewidth=0.5)
            plt.scatter(y=y, x=x, color = col, edgecolors='k', vmax=2, vmin=0)

        
    all_cis.append(cis)
    t_dfm = test.groupby('catid')['i'].max()[:-1]
    [plt.axvline(x=xc, color='k', linestyle='--') for xc in t_dfm]
    plt.title(f'Difference in p-values for males and females for the phenome-wide profiling of mode {k}',fontsize=24)
    plt.ylabel('Average absolute difference in p-values across 1,000 bootstrap iterations', fontsize=14)
    path = f'BONF/mode_{k}_absolute_diff_pvals_err_bars.png'
    plt.savefig(path)
    plt.show()

In [ ]:
#storing results in csv tables
for i in tqdm(range(0,25)):
    
    dfs = pd.DataFrame(columns = ['catid', 'coid', 'phesid', 'phenotype', 'mean_diff_p_val'])
    sig_catids_m = np.genfromtxt(f'MALES_hits_Probability_Mode_{i+1}.txt')
    sig_catids_f = np.genfromtxt(f'FEMALES_hits_Probability_Mode_{i+1}.txt')
    
    phes_m = all_df_m[all_df_m.columns[i::25]].T
    phes_f = all_df_f[all_df_f.columns[i::25]].T
    phes_diff = abs(phes_m-phes_f)
    phes_diff = phes_diff
    
    phes_id = []
    all_infos = []
    
    for j in range (0,977):
        if ((j in sig_catids_m) | (j in sig_catids_f)): 
            phes_id.append(j)
            df_info = pd.DataFrame(test.iloc[j,:3]).T
            phen = df_info.coid.values[0]
            if phen in y_desc_dict.keys():
#                 print(phen)
#                 print(y_desc_dict[phen])
                df_info['phenotype'] = y_desc_dict[phen]
            new_phen = phen.split('_')[0].split('#')[0]
            new_phen = new_phen + '-' + f'{phen.split("_")[-1]}'
            new_new_phen = phen.split('_')[0].split('#')[0] + '-0.0'
            
            if new_phen in y_desc_dict.keys():
                df_info['phenotype'] = y_desc_dict[new_phen]
            
            if new_new_phen in y_desc_dict.keys():
                #print(new_new_phen)
                
                df_info['phenotype'] = y_desc_dict[new_new_phen].split('(')[0] + f'({phen.split("_")[-1]})'
                
            df_info['mean_diff_p_val'] = phes_diff[j].mean()
            df_info['5th_percentile'] = np.percentile(phes_diff.iloc[:,j], 5)
            df_info['25th_percentile'] = np.percentile(phes_diff.iloc[:,j], 25)
            df_info['median'] = np.percentile(phes_diff.iloc[:,j], 50)
            df_info['75th_percentile'] = np.percentile(phes_diff.iloc[:,j], 75)
            df_info['95th_percentile'] = np.percentile(phes_diff.iloc[:,j], 95)
            dfs = dfs.append(df_info)
    dfs.to_csv(f'BONF/mode_{i+1}_pvals_diff_males_females_hits.csv')